<table>
  <tr>
    <td><div align="left"><font size="30">Camera and image motion</font></div></td>
    <td><img src="https://github.com/Tobias-Fischer/RVSS2022/blob/main/Robotic_Vision/common/RVSS-logo.png?raw=1" width="400"></td>
  </tr>
</table>

In [1]:
import numpy as np
from machinevisiontoolbox import CentralCamera
from spatialmath import SE3
import scipy.linalg as lin

%matplotlib inline
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=120, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})


***
# Relationship between camera and image-plane motion
Instantiate a projective camera object, centred at the origin and viewing parallel to the world frame z-axis

In [18]:
camera = CentralCamera.Default()
print(camera)

           Name: default perspective camera [CentralCamera]
     pixel size: 1e-05 x 1e-05
     image size: 1000 x 1000
           pose: t = 0, 0, 0; rpy/yxz = 0°, 0°, 0°
   principal pt: [     500      500]
   focal length: [   0.008    0.008]



Define a point in the world

In [4]:
P = [1, 1, 5]

We will project it to the image plane

In [6]:
p0 = camera.project_point(P)
p0

array([[     660],
       [     660]])

Now if we displace the camera slightly (distance `d`) in the x-direction the pixel coordinates will become

In [8]:
d = 0.1
px = camera.project_point(P, pose=SE3([d, 0, 0]))
px

array([[     644],
       [     660]])

Using the camera coordinate conventions of RVC2 Fig. 11.5, the camera has moved to the right so the image point has moved to the left. The sensitivity of image motion to camera motion is

In [9]:
( px - p0) / d

array([[    -160],
       [       0]])

which is an approximation to the partial derivative $\partial p / \partial x$. It shows that 1 m of camera motion would lead to −160 pixel of feature motion in the u-direction. 

We can repeat this for z-axis translation, ie. moving the camera forward


In [11]:
( camera.project_point(P, pose=SE3([0, 0, d])) - p0) / d

array([[   32.65],
       [   32.65]])

which is an approximation to the partial derivative $\partial p / \partial z$, and  shows equal motion in the u- and v-directions

We can also rotate the camera about the x-axis, equivalent to pitching the camera upwards

In [12]:
px = camera.project_point(P, pose=SE3.Rx(d))
( px - p0) / d

array([[   40.96],
       [   851.9]])

which is predominantly motion down in the image, remember the v-axis points down in the image. the image motion is predominantly in the v-direction. It is clear that camera motion along and about the different degrees of freedom in SE(3) causes quite different motion of image points.

## Things to try on your own
1. use a smaller value of `d` when computing the numerical approximation to the Jacobian

# Image Jacobian

Consider the camera as a function 
\begin{equation}
p = {\cal P}(P, \mathbf{K}, \xi)
\end{equation}
where $p \in \mathbb{R}^2$ is the image-plane point, $P \in \mathbb{R}^3$ is the world point, $\mathbf{K} \in \mathbb{R}^{3 \times 3}$ is the camera intrinsics and $\xi \in SE(3)$ is the camera pose. Its derivative with respect to time is
\begin{equation}
\dot{p} = \mathbf{J}_p(P, \mathbf{K}, \xi) \nu
\end{equation}
where $\nu = (v_x, v_y, v_z, \omega_x, \omega_y, \omega_z) \in \mathbb{R}^6$ is the velocity of the camera -- the spatial velocity.

$\mathbf{J}_p$ is a Jacobian-like object, but because we have taken the derivative with respect to a pose $\xi \in SE(3)$ rather than a vector it is technically called an _interaction matrix_. However in the visual servoing world it is more commonly called an _image Jacobian_ or a _feature sensitivity matrix_.

The Toolbox CentralCamera class provides the method visjac_p to compute the image Jacobian and for the example above it is


In [13]:
J = camera.visjac_p( (672, 672), 5)
J

array([[    -160,        0,     34.4,    36.98,     -837,      172],
       [       0,     -160,     34.4,      837,   -36.98,     -172]])

Each column gives the image-plane velocity for the corresponding component of camera velocity, ie. the first column corresponds to camera x-axis motion, the second column corresponds to camera x-axis motion and the last column correpsonds to camera z-axis rotation.  In Out[6] we computed a numerical approximation to the first column,  Out[7] an approximation to the third column, and Out[8] is an approximation to the fourth column.

If the camera is moving at 1m/s in the x-axis direction the velocity at the pixel will be

In [14]:
J @ [1, 0, 0, 0, 0, 0]

array([    -160,        0])

in units of pixel/s.

If the velocity was 1rad/s around the z-axis the velocity at the pixel will be

In [15]:
J @ [0, 0, 0, 0, 0, 1]

array([     172,     -172])

and if the camera velocity was 1m/s in the x-axis direction *and* 1rad/s in around the z-axis the velocity at the pixel will be

In [16]:
J @ [1, 0, 0, 0, 0, 1]

array([      12,     -172])

# Optical flow fields

We just computed the image Jacobian for a single point close to the centre of the image plane.  Imagine now that we compute it for a grid over the image plane and then see how the image-plane velocity at each point varies with camera veloicty.

The example below provides you with a slider for each component of camera velocity.  As you adjust them the corresponding optical flow field is displayed. Have a play and try to understand what this is showing you.  What happens if you motion in several dimensions of the camera velocity space?

In [19]:
import ipywidgets as widgets

camera = CentralCamera.Default() # create camera object


@widgets.interact
def animate( x =  widgets.FloatSlider(value=0, description='X:',  min=-1, max=1),
             y =  widgets.FloatSlider(value=0,   description='Y:',  min=-1, max=1),
             z =  widgets.FloatSlider(value=0,   description='Z:',  min=-1, max=1),
             rx = widgets.FloatSlider(value=0,   description='rX:', min=-1, max=1),
             ry = widgets.FloatSlider(value=0,   description='rY:', min=-1, max=1),
             rz = widgets.FloatSlider(value=0,   description='rZ:', min=-1, max=1) ):

    Z = 2  # distance to the grid of points
    vel = [x, y, z, rx, ry, rz]  # camera velocity from sliders, as a column vector
    
    # setup the plot window, fixed scale, unity aspect ratio, grid lines on
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    plt.grid(True)
    ax.set_aspect('equal')
    ax.set_facecolor('yellow')
    plt.xlabel('u (pixels)')
    plt.ylabel('v (pixels)')
    plt.title('Velocity on camera image plane')
    plt.xlim(0, 1000)
    ax.set_ylim(1000, 0)  # inverted y-axis


    # compute optical flow over a grid of points
    a = np.arange(-1000, 1000, 50)  # set of flow points

    [U, V] = np.meshgrid(a, a, indexing='ij')
    du = np.empty(U.shape)
    dv = np.empty(U.shape)
    
    for i in range(U.shape[1]):
        for j in range(U.shape[0]):
            pdot = camera.visjac_p( ([U[i,j], V[i,j]]), Z) @ vel
            du[i,j] = pdot[0]
            dv[i,j] = pdot[1]

    # plot the flow vectors
    #  -dv is to overcome a bug with quiver and flipped vertical axis
    plt.quiver(U, V, du, -dv, scale=8000)
    
    

interactive(children=(FloatSlider(value=0.5, description='X:', max=1.0, min=-1.0), FloatSlider(value=0.0, desc…

## Things to try on your own
1. Change the camera to a 4mm focal length, and see the change in the shape of the optical flow fields for rotation about the x- and y-axes.

# Imperceptible motion
Define a point in the world and compute its image Jacobian

In [21]:
P = [0, 0, 5]
p = camera.project_point(P)
J = camera.visjac_p( p, 5)

The Jacobian is $2 \times 6$ which means it has a nullity of 4, that is, there are four possible velocities that will cause zero image plane motion, as will any linear combination of these velocities

In [22]:
N = lin.null_space(J)
N.shape

(6, 4)

We can quickly verify this is the case for the first column which is a camera velocity of

In [23]:
N[:,0]

array([       0,        0,        1,        0,        0,        0])

and the resulting image-plane velocity is

In [24]:
J @ N[:,0]

array([       0,        0])

which is effectively zero.

# Ambiguous motion

In [26]:
P = [1, 1, 5]
p = camera.project_point(P)
J = camera.visjac_p( p, 5)
J

array([[    -160,        0,       32,       32,     -832,      160],
       [       0,     -160,       32,      832,      -32,     -160]])

In [28]:
np.set_printoptions(precision=3, floatmode='maxprec')
np.set_printoptions(suppress=True)
@widgets.interact
def animate( f =  widgets.FloatSlider(value=8, description='f (mm): ',  min=1, max=100) ):
    camera = CentralCamera.Default(f=f*1e-3) # create camera object
    p = camera.project_point(P)
    J = camera.visjac_p( p, 5)
    print(J)

interactive(children=(FloatSlider(value=8.0, description='f (mm): ', min=1.0), Output()), _dom_classes=('widge…